In [1]:
Q = QQ['x']
x = Q.gen()

############################################# INPUT #########################################################

#SETTING OF THE MINIMAL POLYNOMIAL

# f = x**3-x**2-x-1 #d-Bonnaci number
# f = x**2-x-1   #Golden ratio
# f = x**2-3*x+1 #second power of the golden ratio
#f = x**2+4      #2i
# f = x**2+3*x+3 #Eisenstein
f = x**2-2*x-1 #Silver ratio
# f = x**2 +2*x+2 # Penney

#SETTING OF THE ALPHABET
A = [] #alphabet
# A = [0, 1, -1]
A = [0, 1, 2, -1, -2]
# A = [0,1,2,3,-1,-2,-3]
#A = [0,1,2,3,4,5,6,7,-1,-2,-3,-4,-5,-6,-7]
# A = [0,1,2,3,4,5,-1,-2,-3,-4,-5]
# A = [0,1,2,3,4,5,6,7,8,9,1,11,12,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11,-12]


print " v = omega, w = omega^2"
print "Abeceda A = ", A
print ""

###############################################################################################################

print "Je ireducibilni: ", f.is_irreducible() #returns if the polynomial f is ireducible or not

N.<beta> = NumberField(f) #returns a root closes to the embedding
print N #returns what is N and what is beta

roots = beta.complex_embeddings()
print roots #returns roots of the polynomial f

pocetMimoK = 0
pozice = 0
for i in range(f.degree()):
    if abs(roots[i])>=1:
        pocetMimoK = pocetMimoK + 1
        pozice = i
#         print pozice
pisot = 0
kPisot = 0

if (pocetMimoK == 2) & (not roots[pozice].is_real()): #finds out if beta is Pisot or complex Pisot or neither
    print("Beta je komplex Pisot")
    kPisot = 1
    pozice = pozice - 1
elif (pocetMimoK == 1) & roots[pozice].is_real() & (beta > 1):
    print("Beta je Pisot")
    pisot = 1
else: 
    print("Beta neni ani Pisot ani komplex Pisot")
    
print N.places()
# pozice = 0                       #only if N acts unexpectedly
beta_complex = N.places()[pozice]  
print "beta = ", beta_complex(beta)

# A = [0, 1, -1, beta+1, -beta-1] #alphabet for Penney
# A = [0, 1, -1, beta+1, -beta-1, 1/(beta+1), -1/(beta+1)] #alphabet for Eisenstein

print "Abeceda A = ", A

def vytvoritRetezec(k, i): #creates a list from D[k] and digit i
    pomocna = list(SS[k])
    pomocna = list([i] + pomocna[0:pocetPozic-2])
    return list(pomocna) 

def vytvoritPravidlo(prvniList, druhyList): #function which concatenate two lists in rewriting rule
    pList = ""
    dList = ""
    for i in range(1,len(prvniList)+1):
        if prvniList[len(prvniList) - i] == beta + 1:
            pList = pList + "v"
        elif prvniList[len(prvniList) - i] == -beta -1:
            pList = pList + "-v"
        elif prvniList[len(prvniList) - i] == 1/(beta+1):
            pList = pList + "w"
        elif prvniList[len(prvniList) - i] == -1/(beta+1):
            pList = pList + "-w"
        else:
            pList = pList + str(prvniList[len(prvniList) - i])
    for j in range(1,len(druhyList)+1):
        if druhyList[len(druhyList) -j] == beta + 1:
            dList = dList + "v"
        elif druhyList[len(druhyList) -j] == -beta -1:
            dList = dList + "-v"
        elif druhyList[len(druhyList) -j] == 1/(beta+1):
            dList = dList + "w"
        elif druhyList[len(druhyList) -j] == -1/(beta+1):
            dList = dList + "-w"
        else:
            dList = dList + str(druhyList[len(druhyList) -j])
    return "".join(pList) + " -> " + "".join(dList)

def zapsatPravidlo(j, k, i, rad): #writes rewriting rule into appropriate sets 
    pomocna = list(SS[k])
    LL.append(vytvoritPravidlo(list([i] + pomocna[0:rad]),SS[spektrum.index(j)][:rad+1]))
    L.append([beta*spektrum[k]+i, rad+1])
    LonlyPositive.append([beta*spektrum[k]+i, rad+1])

def pridatNovyPrvek(k, i, rad): #adds a new element to the spectrum and checks if it can be rewritten or not
    retezec = vytvoritRetezec(k, i)
    if kontrolaNovehoPrvku(rad, retezec) == True:
        mnozinaNezmensitelnych.append(beta_complex((beta*spektrum[k]+i)/(beta**(rad+1)))) #prida do mnoziny hodnoty 0.abc ...
        retezceNezmensitelnych.append(retezec)
    spektrum.append(beta*spektrum[k]+i) #add of new element into the set spektrum
    SS.append(retezec)

def vymazatZmensitelnyPrvek(index): #delete of the element on the position index from the set mnozinaNezmensitelnych
    mnozinaNezmensitelnych.pop(index)
    retezceNezmensitelnych.pop(index)
       
def kontrolaNovehoPrvku(rad, retezec): #returns TRUE for elements which can not be rewritten and FALSE otherwise
    if len(L)!=0:
        extremy = [[min(column), max(column)] for column in zip(*L)] 
        nejmensiPravidlo = extremy[1][0]
        nejvetsiPravidlo = extremy[1][1]
        for o in range(nejmensiPravidlo, nejvetsiPravidlo +1): #iteration over the size of the rewriting rules
            hodnota = 0
            for p in range(o):
                if rad - p >= 0:
                    hodnota = hodnota * beta + retezec[rad-p]
                else:
                    hodnota = hodnota * beta
            if ([hodnota, o] in L) or ([-hodnota,o] in L):
                return False
    return True

def kontrolaStarychPrvku():#checks if elements in set mnozinaNezmensitelnych can not be rewritten while adding 0 to the end
    pomocne = [i for i in retezceNezmensitelnych]
    for z in pomocne:
        for y in range(pocetPozic-1):
            if z[pocetPozic-2-y]!=0:
                radPrvku = pocetPozic -2- y
                break
        if kontrolaNovehoPrvku(radPrvku, z) == False:
#             print "Stary prvek ", z, " jde zmensit "
            vymazatZmensitelnyPrvek(retezceNezmensitelnych.index(z))
            
def kontrolaNetrivReprezentace(A):#checks if for beta real there exists a non-trivial 0 representation
    print "Kontrola netrivialni reprezentace:"
    minA = min(A)
    maxA = max(A)
    if beta_complex(beta) <= max([maxA+1,-minA+1]):
        print "Ma netriv. reprezentaci"
        return true
    else:
        print "Nema netriv. reprezentaci"
        return false
    
def computeH(A): #program/function to compute an accurate value of parameter H in the algorithm for preprocessing
    global H
    degree = 0
    mnozina = []
    mnozina = list(A)
    pocetPredchozichPrvku = []
    pocetPredchozichPrvku.append(len(A))
    sumaPoctuPredchozich = 0
    retezec = []
    if kPisot == 1:
        for i in range(1, 10):
            print beta_complex(beta**i)
            if beta_complex(beta**i).imag() == 0:
                degree = i
                print "H pocitame do radu beta = ", degree
                break
        pomocna = [0 for i in range(degree +1)]
        for i in A: 
            pomocna[0] = i
            retezec.append(list(pomocna))
        for j in range(1, degree):
            for prvek in range(sumaPoctuPredchozich, pocetPredchozichPrvku[j-1]):
                for a in A:
                    mnozina.append(mnozina[prvek]*beta+a)
            sumaPoctuPredchozich = sumaPoctuPredchozich + pocetPredchozichPrvku[j-1]
            pocetPredchozichPrvku.append(len(mnozina)-sumaPoctuPredchozich)
            print sumaPoctuPredchozich
            print pocetPredchozichPrvku
        nejvetsi = max(map(abs, map(beta_complex, mnozina)))
        H = nejvetsi/(abs(beta_complex(beta**degree))-1) #accurate value of H
        print H
        for i in mnozina: #finding of the largest element which was used in computing of H
            if abs(beta_complex(i)) == nejvetsi:
                print "nejvetsi prvek v mnozine je ", i
    else:                 #if beta is not complex Pisot
        H = max(map(beta_complex, A))/(abs(beta_complex(beta))-1)
    
def getDforNetrivRepres(A):#computes the value of Dmin if there exists only trivial representation of 0
    minA = min(A)
    global D
    D = beta_complex((beta-1+minA)/(beta*(beta-1)))
    print beta_complex((beta-1+minA)/(beta*(beta-1)))
            
def getSpectrumForUptoM(spektrum, A, H, maxRad): #generating of the spectrum upto the power BETA^m
    file = open("pravidla.txt", "w")
    file.write("Pravidla pro polynom " + str(f) + "\n")
    global D
    global epsilon
    global pocetPozic
    global vsechnyLL
    global vsechnyL
    global LL
    global L
    global LonlyPositive
    global LLnewLowerThanD
    global LLlowerThanD
    nulovy = 1
    sumaPrvkuNizsihoRadu = 0
    print "sumaPrvkuNizsihoRadu ", sumaPrvkuNizsihoRadu
    pocetNovychPrvku.append(len(spektrum))
    
    nejmensiRetezec = H
    print "nejmensi retezec je ", beta_complex(nejmensiRetezec)
    
    for rad in range(1,maxRad+1):
        print "pocet prvku ve spektru ", len(spektrum)
        print "pocet novych prvku ", pocetNovychPrvku
        vsechnyLL = vsechnyLL + [i for i in LL if i not in vsechnyLL]
        vsechnyL = vsechnyL + [i for i in L if i not in vsechnyL]
        print "pocet kladnych pravidel :", len(vsechnyLL)
        epsilon = 1000
        print ""
        m = rad +1
        print ""
        print "##########################################################################################"

        print ""
        for k in range(sumaPrvkuNizsihoRadu, sum(pocetNovychPrvku)): #iteration over the existing elements from previous iteration
            for i in A: 
                novy = 1
                for j in spektrum: #comparison of ki with every element of the spectrum
                    if beta*spektrum[k] + i == j: #comparison of the value
                        novy = 0
                        if SS[spektrum.index(j)][rad]==0: #if they are not of the same length, rewriting rule is saved
#                             if (beta_complex(SS[k][rad-1]) >=1) or (beta_complex(SS[k][rad-1] - beta-1) ==0 or beta_complex(SS[k][rad-1]-1/(beta+1))==0): 
#                             if (beta_complex(SS[k][rad-1]) >=1) or (beta_complex(SS[k][rad-1] - beta-1) ==0):
                            if (beta_complex(SS[k][rad-1]) >=1): #only rewriting rules starting on a positive number are saved (if the alphabet is complex, uncomment previous)
                                zapsatPravidlo(j, k, i, rad)
                            elif SS[k][rad-1]<0:
                                L.append([beta*spektrum[k]+i, rad+1])
                        break
                if novy == 1:
                    if abs(beta_complex(beta*spektrum[k]+i)) !=0:
#                             print abs(beta_complex(beta*spektrum[k]+i))- H
                        if (SS[k][rad-1]!=0) and (abs(beta_complex(beta*spektrum[k]+i)) - H < epsilon):
                            epsilon = abs(beta_complex(beta*spektrum[k]+i))- H
                            print "meni se na ", epsilon
                            print SS[k], " + ", i
#                             else:
#                                 print SS[k], " + ", i, " je mimo kruznici, hodnota ", abs(beta_complex(beta*spektrum[k]+i)), abs(beta_complex((beta*spektrum[k]+i)/(beta**(rad+1))))
                        pridatNovyPrvek(k, i, rad)
        kontrolaStarychPrvku()                      
        sumaPrvkuNizsihoRadu = sum(pocetNovychPrvku) #actual number of elements from previous iterations
        pocetNovychPrvku.append(len(spektrum)-sumaPrvkuNizsihoRadu) #adding of the number of elements from the last iteration
        print ""
        print "epsilon = ", epsilon, " m = ", m
        if epsilon < 0: #if epsilon is <0, Dmin will be <0 -> one more iteration
            print "NEPLATNE D !!!!!!!!!!"
        else:
            print "PLATNE D"

        D = min(min(map(abs, mnozinaNezmensitelnych)), epsilon/(abs(beta_complex((beta)^m)))) #checks if rewriting rules can be omitted or not (if they have value greater than Dmin)
        LLnewLowerThanD = [i for i in LL if ((abs(beta_complex(LonlyPositive[LL.index(i)][0]/(beta**LonlyPositive[LL.index(i)][1]))) - nejmensiRetezec/abs(beta_complex(beta**L[LL.index(i)][1]))<=D) and (i not in LLlowerThanD))]
        print LLnewLowerThanD
        LLlowerThanD = LLlowerThanD + LLnewLowerThanD
        print LLlowerThanD
            
        print "Pocet prepisovaci pravidel mensich nez Dmin je: ", len(LLlowerThanD)
            
        print ""
        print "Pocet prepisovacich pravidel je:", len(LL) #total number of rewriting rules found until now
        file.write("\n" + "m = %d \n \n" % m)
        temp = [i for i in LL if (i not in vsechnyLL)] #writes all new rewriting rules for the power m
        for i in temp:
            print(i)
            file.write(i + "\n") #writes rewriting rule into the file
        print ""
        
        print "m = ", m
        print "hodnota D = ", D
        print ""
        nejmensiPrvek = min(map(abs, mnozinaNezmensitelnych))
        print " min(map(abs, mnozinaNezmensitelnych)) = ", nejmensiPrvek
        print " epsilon/(abs(beta)^m)) = ",epsilon, " / abs(beta)^", m , " = ", epsilon/(abs(beta_complex((beta)^m)))  

        for i in mnozinaNezmensitelnych:
            if abs(i) == nejmensiPrvek:
                print "nejmensi prvek v mnozine nezmensitelnych je ", retezceNezmensitelnych[mnozinaNezmensitelnych.index(i)]
    file.close()    

print ""
if (pisot == 1) or (kPisot==1): 
    print "max z abecedy je ", max(map(beta_complex,A))
    print "hodnota H:"
    computeH(A)
    print "hruby odhad by byl ", max(map(beta_complex, A))/(abs(beta_complex(beta))-1) #comparison with a rough estimate of H
    print "H = ", H ," s presnou hodnotou ", H

    spektrum = [] #set of elements of the spectrum
    SS = [] #set of values of the elements of the spectrum

########################################

    pocetPozic = 12 #bound for number of digits in the strings
    radBeta = pocetPozic -1

########################################

    mnozinaNezmensitelnych = [] #list of elements of spectrum which are inside the circle H + epsilon which can not be rewritten
    retezceNezmensitelnych = []
    pomocna = [0 for i in range(pocetPozic)]
    for k in A:
        if k != 0: #we do not consider 0 
            pomocna[0] = k
            retezceNezmensitelnych.append(list(pomocna))
            mnozinaNezmensitelnych.append(beta_complex(k / beta))

    pomocna = [0 for i in range(pocetPozic)]
    for i in A: #adds alphabet to the sets
#         if i >= 0:
        pomocna[0] = i
        spektrum.append(i)
        SS.append(list(pomocna))
    
    LL = [] #list of rewriting rules
    L = [] #list of values of every rewriting rule (even starting on a negative digit)
    LonlyPositive = [] #list of values of elements of LL
    LLnewLowerThanD = [] #list of new rewriting rules smaller than Dmin
    LLlowerThanD = [] #list of all rewriting rules smaller than Dmin
    vsechnyL = []
    vsechnyLL = []
    pocetNovychPrvku = []

    if (kontrolaNetrivReprezentace(A) == true) or (kPisot ==1):
        getSpectrumForUptoM(spektrum, A, H, 5)
    else: 
        D = 0
        getDforNetrivRepres(A)
        
        print "hodnota D = ", D
            
print "FINISHED"

 v = omega, w = omega^2
Abeceda A =  [0, 1, 2, -1, -2]

Je ireducibilni:  True
Number Field in beta with defining polynomial x^2 - 2*x - 1
[-0.414213562373095, 2.41421356237309]
Beta je Pisot
[Ring morphism:
  From: Number Field in beta with defining polynomial x^2 - 2*x - 1
  To:   Real Field with 106 bits of precision
  Defn: beta |--> -0.4142135623730950488016887242097, Ring morphism:
  From: Number Field in beta with defining polynomial x^2 - 2*x - 1
  To:   Real Field with 106 bits of precision
  Defn: beta |--> 2.414213562373095048801688724210]
beta =  2.414213562373095048801688724210
Abeceda A =  [0, 1, 2, -1, -2]

max z abecedy je  2.000000000000000000000000000000
hodnota H:
hruby odhad by byl  1.414213562373095048801688724210
H =  1.414213562373095048801688724210  s presnou hodnotou  1.414213562373095048801688724210
Kontrola netrivialni reprezentace:
Ma netriv. reprezentaci
sumaPrvkuNizsihoRadu  0
nejmensi retezec je  1.414213562373095048801688724210
pocet prvku ve spektru  5



epsilon =  31.97056274847714058562026469052  m =  6
PLATNE D
['1-22-220 -> 00211-1', '1-22-22-1 -> 00211-2']
['1-1-1 -> 010', '1-1-2 -> 01-1', '1-20 -> 001', '1-21 -> 002', '1-2-1 -> 000', '1-2-2 -> 00-1', '1-220 -> 011-1', '1-10 -> 011', '1-221 -> 0110', '1-22-1 -> 011-2', '1-22-20 -> 00122', '1-22-21 -> 00202', '1-22-2-1 -> 00121', '1-22-2-2 -> 00120', '1-22-220 -> 00211-1', '1-22-22-1 -> 00211-2']
Pocet prepisovaci pravidel mensich nez Dmin je:  16

Pocet prepisovacich pravidel je: 90
1021-10 -> 022221
1021-11 -> 022222
1021-1-1 -> 022220
1021-1-2 -> 02222-1
1021-20 -> 022211
1021-21 -> 022212
1021-2-1 -> 022210
1021-2-2 -> 02221-1
102-120 -> 02221-1
102-121 -> 022210
102-122 -> 022211
102-12-1 -> 02221-2
102-220 -> 02211-1
102-221 -> 022110
102-222 -> 022111
102-22-1 -> 02211-2
11-12-20 -> 022122
11-12-21 -> 022202
11-12-2-1 -> 022121
11-12-2-2 -> 022120
11-22-20 -> 021122
11-22-21 -> 021202
11-22-2-1 -> 021121
11-22-2-2 -> 021120
1-12-220 -> 01211-1
1-12-221 -> 012110
1-12-222 ->